# Task C. Processing Data Stream
## Simulating real-time data using Apache Kafka Producers (TERRA DATA)
## Task C - 1(c) Event Producer 3

### Importing Libraries and Reading the CSV 

In [1]:
#importing required libraries
import csv  # to read the csv files
import random  #for random number generation
import json  #for json dumps
import time  #for sleep function
import datetime  #creating datetime deltas
from kafka import KafkaConsumer, KafkaProducer  # creating kafka producers and consumers

In [2]:
def read_csv(file_name):    #function to read the csv file and return the header and data
    header = []
    data = []
    with open(file_name, 'r') as f:
        reader = csv.reader(f)
        row_count = 0
        
        for row in reader:
            if(row_count == 0):
                header.extend(row)
            else:
                data.append(row)
            row_count += 1
    
    return header, data


        
header_list_terra, data_terra = read_csv('hotspot_TERRA_streaming.csv')    #Header and data collected seperately

### Kafka producer

In [3]:
#kafka producer, publish the message into kafka queue in given topic
def publish_message(producer_instance, topic_name, key, value): #Function to publish the randomly generated message
    try:
        #Converting Key and value to Bytes
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        #Creating instance
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        #Flushing
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))

# connects to running kafka instance at port 9092 locally
def connect_kafka_producer():
    _producer = None
    try:
         #connecting to localhost Kafka and the default port
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

### Creating Kafka Connection and Sending Random instance of Climate data
* Using random number generator, randomly selecting a row in TERRA_streaming file and feeding into kafka producer.

* Publish the messages with a key as `TERRA` and value as TERRA data one row at a time.

In [4]:
#kafka producer
#assigning senders
SENDER = 'event3'
SENDER1 = 'event3_3'

while True:
    rnd_num = random.randint(0,len(data_terra)-1)  #Create random numbers
    d = data_terra[rnd_num]#Assign random data
    
    terra_dict = {}#empty dictionary
    # assigning value to the dictionary
    terra_dict['location'] = {'type':"Point", 'coordinates':[float(d[0]),float(d[1])]}
    terra_dict['confidence'] = int(d[2])
    terra_dict['surface_temperature'] = float(d[3])
    terra_dict['sender_id'] = SENDER if rnd_num%2 == 0 else SENDER1
    terra_dict['creation_time'] = str(datetime.datetime.now())
    terra_dict['topic'] = 'TERRA'  #Assigning the Source of data for future reference
    
    #send data to kafka
    
    kafka_producer = connect_kafka_producer()
    key = 'TERRA'
    print(json.dumps(terra_dict))
    publish_message(kafka_producer, 'TERRA', key, json.dumps(terra_dict))
    
    time.sleep(25)  ##wait 25 seconds
    

{"confidence": 67, "surface_temperature": 45.0, "topic": "TERRA", "location": {"type": "Point", "coordinates": [-37.7615, 147.9169]}, "sender_id": "event3", "creation_time": "2019-05-23 10:11:16.006787"}
Message published successfully.
{"confidence": 60, "surface_temperature": 40.0, "topic": "TERRA", "location": {"type": "Point", "coordinates": [-36.0841, 141.1426]}, "sender_id": "event3_3", "creation_time": "2019-05-23 10:11:41.249114"}
Message published successfully.
{"confidence": 62, "surface_temperature": 33.0, "topic": "TERRA", "location": {"type": "Point", "coordinates": [-37.602, 149.295]}, "sender_id": "event3", "creation_time": "2019-05-23 10:12:06.305338"}
Message published successfully.
{"confidence": 67, "surface_temperature": 53.0, "topic": "TERRA", "location": {"type": "Point", "coordinates": [-35.554, 143.307]}, "sender_id": "event3", "creation_time": "2019-05-23 10:12:31.333266"}
Message published successfully.
{"confidence": 96, "surface_temperature": 78.0, "topic": "

{"confidence": 59, "surface_temperature": 40.0, "topic": "TERRA", "location": {"type": "Point", "coordinates": [-37.6868, 142.3949]}, "sender_id": "event3_3", "creation_time": "2019-05-23 10:25:52.781870"}
Message published successfully.
{"confidence": 73, "surface_temperature": 47.0, "topic": "TERRA", "location": {"type": "Point", "coordinates": [-36.2377, 141.1427]}, "sender_id": "event3_3", "creation_time": "2019-05-23 10:26:17.831969"}
Message published successfully.
{"confidence": 85, "surface_temperature": 59.0, "topic": "TERRA", "location": {"type": "Point", "coordinates": [-36.5722, 146.7572]}, "sender_id": "event3_3", "creation_time": "2019-05-23 10:26:42.882618"}
Message published successfully.
{"confidence": 65, "surface_temperature": 43.0, "topic": "TERRA", "location": {"type": "Point", "coordinates": [-36.3757, 145.5589]}, "sender_id": "event3_3", "creation_time": "2019-05-23 10:27:07.923668"}
Message published successfully.
{"confidence": 96, "surface_temperature": 78.0, 

KeyboardInterrupt: 

In [5]:
if kafka_producer is not None:
    kafka_producer.close()